In [1]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE
from src.model.tensorflow_utils import load_dataset, apply_augmentation_to_dataset, oversample_minority_classes, optimize_dataset, count_dataset_size, get_num_classes
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_loss_monitor, setup_garbage_collector, get_metrics
from src.model.tensorflow_utils import calculate_class_weights, show_class_weights, start_or_resume_training, analyze_class_distribution, filter_no_finding_class
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121
from src.model.loss.tensorflow_no_finding_binary_crossentropy import NoFindingBinaryCrossentropy

# Input Data
initial_epoch   = 25
resume_training = True
checkpoint_path = '/Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/checkpoints/cp-0024.keras'
model_name      = "DenseNet121_v5_2"
num_classes     = get_num_classes()

In [2]:
train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1744609000.268217 64410239 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1744609000.268548 64410239 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Optimize Dataset for rare clasess
train_ds = filter_no_finding_class(train_ds)
val_ds   = filter_no_finding_class(val_ds)

class_weights = calculate_class_weights(train_ds, num_classes)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, num_classes)

In [4]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Training
train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

# Validation
val_ds  = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 104233 samples [49:28, 35.12 samples/s]   
Counting samples: 15391 samples [00:16, 927.85 samples/s]


In [5]:
# Setup Model Deps
# Setup Loggers
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
loss_monitor      = setup_loss_monitor(MODELS_PATH, model_name, logger, val_ds, resume_training=resume_training, initial_epoch=initial_epoch)
garbage_collector = setup_garbage_collector(logger)
metrics           = get_metrics()

# Setup compile arguments
loss       = BinaryCrossentropy(from_logits=False, label_smoothing=0.01)
reduce_lr  = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-04-14 09:24:28 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/train_metrics.csv
2025-04-14 09:24:28 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/val_metrics.csv
2025-04-14 09:24:28 - INFO - Cleaned training metrics file, kept 78168 records before epoch 25
2025-04-14 09:24:28 - INFO - Cleaned validation metrics file, kept 24 records before epoch 25
2025-04-14 09:24:28 - INFO - Found 78168 existing training records
2025-04-14 09:24:28 - INFO - Found 24 existing validation records
2025-04-14 09:24:28 - INFO - Cleaned loss analysis metrics file, kept 336 records before epoch 25
2025-04-14 09:24:28 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/loss_analysis_metrics.csv


In [6]:
# Model Training
model          = build_densenet121(num_classes, use_se=True, use_sa=True)
optimizer      = Adam(learning_rate=1e-4, clipnorm=1.0)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    30,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, loss_monitor, garbage_collector], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-04-14 09:24:30 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/train_metrics.csv
2025-04-14 09:24:30 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/val_metrics.csv
2025-04-14 09:24:30 - INFO - Cleaned training metrics file, kept 78168 records before epoch 25
2025-04-14 09:24:30 - INFO - Cleaned validation metrics file, kept 24 records before epoch 25
2025-04-14 09:24:30 - INFO - Found 78168 existing training records
2025-04-14 09:24:30 - INFO - Found 24 existing validation records
2025-04-14 09:24:30 - INFO - Cleaned loss analysis metrics file, kept 336 records before epoch 25
2025-04-14 09:24:30 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/loss_analysis_metrics.csv


Loading full model from checkpoint: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v5_2/checkpoints/cp-0024.keras


2025-04-14 09:24:34 - INFO - 
=== Training Started ===

2025-04-14 09:24:34 - INFO - Batch Size: 32
2025-04-14 09:24:34 - INFO - Optimizer: Adam
2025-04-14 09:24:34 - INFO - 

2025-04-14 09:24:34 - INFO - 
=== Starting Epoch 25 ===



Epoch 25/30
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 0s 722ms/step - accuracy: 0.9227 - auc: 0.7727 - f1_score: 0.1274 - loss: 0.1988 - precision: 0.5254 - recall: 0.0899   

2025-04-14 10:07:16 - INFO - 
=== Epoch 25 Summary ===
2025-04-14 10:07:16 - INFO - Time: 2562.41s
2025-04-14 10:07:16 - INFO - Training   - accuracy: 0.9229 - auc: 0.7720 - f1_score: 0.1245 - loss: 0.1995 - precision: 0.5240 - recall: 0.0875 - learning_rate: 0.0001
2025-04-14 10:07:16 - INFO - Validation - accuracy: 0.9372 - auc: 0.7537 - f1_score: 0.0797 - loss: 0.2218 - precision: 0.2440 - recall: 0.0598
2025-04-14 10:07:16 - INFO - ==================================================



...

2025-04-14 14:42:42 - INFO - 
=== Epoch 30 Summary ===
2025-04-14 14:42:42 - INFO - Time: 3299.14s
2025-04-14 14:42:42 - INFO - Training   - accuracy: 0.9232 - auc: 0.7810 - f1_score: 0.1444 - loss: 0.1968 - precision: 0.5292 - recall: 0.1019 - learning_rate: 0.0001
2025-04-14 14:42:42 - INFO - Validation - accuracy: 0.9328 - auc: 0.7580 - f1_score: 0.1304 - loss: 0.2347 - precision: 0.2523 - recall: 0.1045
2025-04-14 14:42:42 - INFO - ==================================================



1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-04-14 14:44:42 - INFO - 
Loss Analysis - Atelectasis
2025-04-14 14:44:42 - INFO - Confidence Distribution:
2025-04-14 14:44:42 - INFO - -- High (>0.9): 0.00%
2025-04-14 14:44:42 - INFO - -- Medium (0.6-0.9): 0.00%
2025-04-14 14:44:42 - INFO - -- Uncertain (0.4-0.6): 0.08%
2025-04-14 14:44:42 - INFO - -- Low (<0.4): 99.92%
2025-04-14 14:44:42 - INFO - Performance:
2025-04-14 14:44:42 - INFO - -- True Positives: 2
2025-04-14 14:44:42 - INFO - -- False Positives: 1
2025-04-14 14:44:42 - INFO - -- Loss Contribution: 0.3390
2025-04-14 14:44:42 - INFO - Average Confidence:
2025-04-14 14:44:42 - INFO - -- Correct Predictions: 5.26%
2025-04-14 14:44:42 - INFO - -- Incorrect Predictions: 9.99%
2025-04-14 14:44:42 - INFO - 
Loss Analysis - Cardiomegaly
2025-04-14 14:44:42 - INFO - Confidence Distribution:
2025-04-14 14:44:42 - INFO - -- High (>0.9): 0.36%
2025-04-14 14:44:42 - INFO - -- Medium (0.6-0.9): 2.23%
2025-04-14 14:44:42 - INFO - -- Uncertain (0.4-0.6): 2.53%
2025-04-14 14:44:42 - 

3257/3257 ━━━━━━━━━━━━━━━━━━━━ 3631s 1s/step - accuracy: 0.9231 - auc: 0.7781 - f1_score: 0.1461 - loss: 0.1974 - precision: 0.5361 - recall: 0.1035 - val_accuracy: 0.9328 - val_auc: 0.7580 - val_f1_score: 0.1304 - val_loss: 0.2347 - val_precision: 0.2523 - val_recall: 0.1045 - learning_rate: 1.0000e-04


2025-04-14 14:48:14 - INFO - 
=== Training Completed! ===

2025-04-14 14:48:14 - INFO - Final Metrics: accuracy: 0.9232 - auc: 0.7810 - f1_score: 0.1444 - loss: 0.1968 - precision: 0.5292 - recall: 0.1019 - val_accuracy: 0.9328 - val_auc: 0.7580 - val_f1_score: 0.1304 - val_loss: 0.2347 - val_precision: 0.2523 - val_recall: 0.1045

